In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rd
from scipy.optimize import minimize_scalar
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I

In [2]:
seed = 11667
n_test = 200

m = ModelIntv()

folders = [
    'dy_ladd',
#     'dy_add', 
#     'dy_mul'
]

In [3]:
tar = json.load(open('data/targets_acf.json', 'r'))
tar_mdu = tar['MDU'] / tar['Pop']
tar_d2d = tar['D2D'] / tar['Pop']
tar_mdu, tar_d2d

(0.00014333333333333334, 3.483333333333333e-05)

In [4]:
intv_mdu = {'Year0': 2022, 'ScreenAlg': 'SyCx'}
intv_d2d = {'Year0': 2022, 'ScreenAlg': 'Sy'}

In [5]:
def find_bg_coverage(pars_post):
    def fn(x, src, yld, y0, p):
        intv = {'FullACF': {'Coverage': x, 'Year0': src['Year0'], 'ScreenAlg': src['ScreenAlg']}}
        _, (ms, _), _ = m.simulate_onward(y0, p, intv, polished=True)
        return (ms.ACF_Yield[2022] - yld) ** 2


    x_mdu, x_d2d = np.zeros(len(pars_post)), np.zeros(len(pars_post))

    for i, p in enumerate(pars_post):
        y0, p = m.find_baseline(p)
        sol = minimize_scalar(fn, bounds=(0, 0.5), 
                              args=(intv_mdu, tar_mdu, y0, p, ), method='bounded')
        x_mdu[i] = sol.x

        sol = minimize_scalar(fn, bounds=(0, 0.5), 
                              args=(intv_d2d, tar_d2d, y0, p, ), method='bounded')
        x_d2d[i] = sol.x

    x_mdu = np.quantile(x_mdu, [0.025, 0.5, 0.975])
    x_d2d = np.quantile(x_d2d, [0.025, 0.5, 0.975])

    mdu_l = {'Coverage': x_mdu[0], 'Year0': intv_mdu['Year0'], 'ScreenAlg': intv_mdu['ScreenAlg']}
    mdu_m = {'Coverage': x_mdu[1], 'Year0': intv_mdu['Year0'], 'ScreenAlg': intv_mdu['ScreenAlg']}
    mdu_u = {'Coverage': x_mdu[2], 'Year0': intv_mdu['Year0'], 'ScreenAlg': intv_mdu['ScreenAlg']}
    
    d2d_l = {'Coverage': x_d2d[0], 'Year0': intv_d2d['Year0'], 'ScreenAlg': intv_d2d['ScreenAlg']}
    d2d_m = {'Coverage': x_d2d[1], 'Year0': intv_d2d['Year0'], 'ScreenAlg': intv_d2d['ScreenAlg']}
    d2d_u = {'Coverage': x_d2d[2], 'Year0': intv_d2d['Year0'], 'ScreenAlg': intv_d2d['ScreenAlg']}
    return {
        'MDU': {'L': mdu_l, 'M': mdu_m, 'U': mdu_u}, 
        'D2D': {'L': d2d_l, 'M': d2d_m, 'U': d2d_u}
    }


In [6]:
for folder in folders:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    sel = rd.choice(list(range(len(pars_post))), n_test)
    sel.sort()
    pars_post = [pars_post[i] for i in sel]
    
    cov_bg = find_bg_coverage(pars_post)

    with open(f'{out_path}/CoverageBg.json', 'w') as f:
        json.dump(cov_bg, f)

dy_ladd


In [7]:
cov_bg

{'MDU': {'L': {'Coverage': 0.007281449372048281,
   'Year0': 2022,
   'ScreenAlg': 'SyCx'},
  'M': {'Coverage': 0.0146291137642422, 'Year0': 2022, 'ScreenAlg': 'SyCx'},
  'U': {'Coverage': 0.03343824885178595, 'Year0': 2022, 'ScreenAlg': 'SyCx'}},
 'D2D': {'L': {'Coverage': 0.010430593136556218,
   'Year0': 2022,
   'ScreenAlg': 'Sy'},
  'M': {'Coverage': 0.017848157250812055, 'Year0': 2022, 'ScreenAlg': 'Sy'},
  'U': {'Coverage': 0.032307994466955405, 'Year0': 2022, 'ScreenAlg': 'Sy'}}}